In [1]:
import os

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.types import ARRAY, Integer, String

In [2]:
df_all_movies = pd.read_csv("./custom_csv/movies_cleaned.csv")
df_netflix_movies = pd.read_csv("./custom_csv/movies_combined.csv")

In [3]:
with open("./similarity_genre.npy", "rb") as f:
    similarity_genres = np.load(f)

with open("./similarity_embedding_keywords.npy", "rb") as f:
    similarity_embedding_keywords = np.load(f)

with open("./similarity_embedding_overview.npy", "rb") as f:
    similarity_embedding_overview = np.load(f)

In [4]:
df_genres = (
    (
        pd.DataFrame(
            np.round(similarity_genres, 2),
            columns=df_netflix_movies.id.to_numpy(),
            index=df_all_movies.id.to_numpy(),
        ).rename_axis("id")
        * 100
    )
    .round()
    .astype(int)
)
df_overview = (
    (
        pd.DataFrame(
            np.round(similarity_embedding_overview, 2),
            columns=df_netflix_movies.id.to_numpy(),
            index=df_all_movies.id.to_numpy(),
        ).rename_axis("id")
        * 100
    )
    .round()
    .astype(int)
)
df_keywords = (
    (
        pd.DataFrame(
            np.round(similarity_embedding_keywords, 2),
            columns=df_netflix_movies.id.to_numpy(),
            index=df_all_movies.id.to_numpy(),
        ).rename_axis("id")
        * 100
    )
    .round()
    .astype(int)
)

In [5]:
df_all_movies = df_all_movies.loc[:, ["id", "title_dated"]]
df_all_movies.rename(columns={"title_dated": "title"}, inplace=True)

In [6]:
df_genres.index = df_genres.index.astype(str)
df_overview.index = df_overview.index.astype(str)
df_keywords.index = df_keywords.index.astype(str)
df_all_movies.id = df_all_movies.id.astype(str)

In [7]:
# df_all_movies.to_csv("./custom_csv/titles.csv", index=False)
# df_genres.to_csv("./custom_csv/genres.csv")
# df_keywords.to_csv("./custom_csv/kw.csv")
# df_overview.to_csv("./custom_csv/ow.csv")

In [8]:
df_table = pd.DataFrame(df_all_movies.id.to_numpy().astype(str), columns=["id"])

In [9]:
df_table["genre"] = 0
df_table["keywords"] = 0
df_table["overview"] = 0
df_table["genre"] = df_table.loc[:, ["genre"]].apply(
    lambda x: df_genres.iloc[x.name, :].tolist(), axis=1
)
df_table["keywords"] = (
    df_table.loc[:, ["keywords"]]
    .apply(lambda x: df_keywords.iloc[x.name, :].tolist(), axis=1)
    .tolist()
)
df_table["overview"] = (
    df_table.loc[:, ["overview"]]
    .apply(lambda x: df_overview.iloc[x.name, :].tolist(), axis=1)
    .tolist()
)

In [10]:
df_all_movies.columns

Index(['id', 'title'], dtype='object')

In [11]:
df_table["title"] = df_all_movies["title"]

In [12]:
# df_table.head()

In [19]:
load_dotenv("./.env")
engine = create_engine(os.environ["DATABASE_URL"])

In [20]:
df_table = df_table.loc[:, ["id", "title", "genre", "keywords", "overview"]]

In [21]:
df_table.head()

,id,title,genre,keywords,overview
0,315946,Passage of Venus,"[71, 58, 71, 58, 71, 100, 0, 0, 0, 0, 0, 0, 0,...","[7, 7, 7, 33, 29, 11, 21, 7, 9, 7, 30, 6, 29, ...","[30, 16, 8, 1, 10, 2, 4, 47, 11, 10, 27, -4, 4..."
1,194079,Sallie Gardner at a Gallop,"[71, 58, 71, 58, 71, 100, 0, 0, 0, 0, 0, 0, 0,...","[7, 7, 7, 33, 30, 6, 37, 7, 27, 7, 37, 23, 46,...","[35, 10, 19, 17, 21, 18, 11, 32, 7, 7, 31, 2, ..."
2,426903,Buffalo Running,"[71, 58, 71, 58, 71, 100, 0, 0, 0, 0, 0, 0, 0,...","[13, 13, 13, 19, 16, 13, 33, 13, 29, 13, 27, 3...","[25, 14, 22, 16, 11, 16, 7, 0, 5, 4, 1, 8, 2, ..."
3,159897,Man Walking Around a Corner,"[71, 58, 71, 58, 71, 100, 0, 0, 0, 0, 0, 0, 0,...","[17, 17, 17, 19, 14, 13, 25, 17, 26, 17, 11, 1...","[29, 11, 26, 12, 21, 15, 7, 18, 4, 13, 22, 2, ..."
4,96882,Accordion Player,"[71, 58, 71, 58, 71, 100, 0, 0, 0, 0, 0, 0, 0,...","[6, 6, 6, 27, 22, 13, 19, 6, 5, 6, 8, 4, 14, 6...","[30, 13, 30, 17, 26, 23, 17, 19, 10, 22, 29, 1..."


In [ ]:
df_table.to_sql(
    "movies_scores",
    engine,
    # method="multi",
    index=False,
    if_exists="replace",
    dtype={
        "id": String,
        "title": String,
        "genre": ARRAY(Integer),
        "keywords": ARRAY(Integer),
        "overview": ARRAY(Integer),
    },
)

In [22]:
df_netflix_movies = df_netflix_movies[["id", "title_dated", "vote_average"]]

In [23]:
df_netflix_movies = df_netflix_movies.rename(
    columns={"title_dated": "title", "vote_average": "review_score"}
)

In [24]:
df_netflix_movies["review_score"] = (df_netflix_movies["review_score"] * 10).astype(int)

In [25]:
df_netflix_movies

,id,title,review_score
0,56143,The Battle of Midway,47
1,112912,Why We Fight: The Battle of Russia,50
2,257084,Tunisian Victory,0
3,151731,The Negro Soldier,60
4,84701,San Pietro,64
...,...,...,...
1718,428493,God's Own Country,54
1719,271404,Beyond Skyline,0
1720,284053,Thor: Ragnarok,0
1721,413992,Sweet Virginia,0


In [27]:
df_netflix_movies.to_sql(
    "netflix",
    engine,
    method="multi",
    index=True,
    if_exists="replace",
    dtype={"index": Integer, "id": String, "title": String, "vote_average": Integer},
)

-1

In [28]:
with engine.connect() as con:
    stuff = con.execute(
        text(
            """
                        SELECT keywords, genre, overview
                        FROM movies_scores
                        WHERE movies_scores.id = '315946'
                        """
        )
    )

In [29]:
test = np.array(stuff.fetchone())

In [30]:
top_ten = list(np.sum(test, axis=0).argsort()[-10:].astype(str))

In [31]:
top_ten

['1507', '1279', '1201', '1652', '1298', '1565', '1358', '1236', '548', '1489']

In [32]:
test[0, list(np.sum(test, axis=0).argsort()[-10:])]

array([24, 19, 28, 26, 33, 23, 27, 19, 41, 30])

In [33]:
with engine.connect() as con:
    stuff = con.execute(
        text(
            f"""
                        SELECT title
                        FROM netflix
                        WHERE netflix.index in ({",".join(top_ten)})
                        """
        )
    )

In [34]:
stuff.all()

[('The Pixar Story',),
 ('Raiders!: The Story of the Greatest Fan Film Ever Made',),
 ('Catching the Sun',),
 ('Miss Sharon Jones!',),
 ('Elstree 1976',),
 ('Sky Ladder: The Art of Cai Guo-Qiang',),
 ('Into the Inferno',),
 ('Silicon Cowboys',),
 ('Harry Benson: Shoot First',),
 ('Chasing Coral',)]